# TNAD Colab Runner (GPU)

End-to-end notebook for executing the Tensor Network-Augmented Decoding (TNAD) experiments on Google Colab with CUDA-enabled GPUs. Follow the cells sequentially.

## 1. Runtime Preparation

1. In Colab, switch the runtime to **GPU**: `Runtime → Change runtime type → Hardware accelerator → GPU`.
2. Execute the cells below in order. Installation steps may take a few minutes the first time.

In [ ]:
# Verify that a CUDA GPU is visible to PyTorch.
import torch

print(f"PyTorch version : {torch.__version__}")
print(f"CUDA available  : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_properties(0)
    print(f"GPU name        : {gpu.name}")
    print(f"Total memory    : {gpu.total_memory / 1e9:.2f} GB")
else:
    raise EnvironmentError("A CUDA-enabled runtime is required. Switch the Colab runtime to GPU.")

In [ ]:
%%capture
!pip install -U pip
!pip install -q protobuf==3.20.3 accelerate bitsandbytes datasets loguru matplotlib pyyaml seaborn sentencepiece tqdm transformers

## 2. Retrieve the Project

Clones the repository that contains the TNAD source code. Update `REPO_URL` if your fork lives at a different location. If you have already uploaded the files manually, you can skip this cell.

In [ ]:
import os
from pathlib import Path

REPO_URL = "https://github.com/your-org/quantum-search-llm.git"  # TODO: replace with your repository URL
REPO_DIR = Path("quantum-search-llm")

if REPO_DIR.exists():
    print(f"Repository already present at {REPO_DIR.resolve()}")
else:
    !git clone --depth 1 {REPO_URL}
    print("Clone complete.")

os.chdir(REPO_DIR)
print(f"Working directory: {Path.cwd()}")

In [ ]:
%%capture
# Install the project in editable mode so notebooks can import the TNAD package.
!pip install -e .

## 3. Configure Experiment Parameters

Adjust the variables below to control which model runs, how many benchmark samples are evaluated, and the FGBS hyperparameters. The defaults are chosen to fit comfortably within a Colab T4/A100 session while still exercising the full pipeline.

In [ ]:
from pathlib import Path
import yaml

MODEL_NAME = "microsoft/phi-2"          # Use a 2.7B model that fits easily on Colab GPUs.
USE_8BIT = True                         # Requires bitsandbytes (installed above). Keeps VRAM usage low for larger models.
ALPHA = 0.5                             # Fluency vs coherence balance.
BOND_DIM = 8                            # Logical bandwidth χ (REDUCED from 16 to 8 for low-memory GPUs)
BEAM_WIDTH = 3                          # Number of beams (REDUCED from 5 to 3 for low-memory GPUs)
NUM_EXAMPLES = 25                       # Evaluate this many samples per benchmark (set to -1 for full dataset).

BASE_CONFIG_PATH = Path("configs/default.yaml")
COLAB_CONFIG_PATH = Path("configs/colab_autogen.yaml")

with BASE_CONFIG_PATH.open("r") as f:
    config = yaml.safe_load(f)

config["model"]["name"] = MODEL_NAME
config["model"]["device"] = "cuda"
config["model"]["load_in_8bit"] = bool(USE_8BIT)
config["model"]["torch_dtype"] = "float16"

config["fgbs"]["alpha"] = float(ALPHA)
config["fgbs"]["bond_dim"] = int(BOND_DIM)
config["fgbs"]["beam_width"] = int(BEAM_WIDTH)
config["fgbs"]["top_k"] = 30  # Reduced from 50 for memory efficiency

config["experiment"]["num_examples"] = int(NUM_EXAMPLES)
config["generation"]["max_length"] = 256  # Reduced from 512 for memory efficiency

# Explicitly enable self-consistency with proper sampling configuration
if "baselines" not in config:
    config["baselines"] = {}
if "self_consistency" not in config["baselines"]:
    config["baselines"]["self_consistency"] = {}

config["baselines"]["self_consistency"]["num_samples"] = 10
config["baselines"]["self_consistency"]["temperature"] = 0.7
print("✓ Self-consistency enabled: num_samples=10, temperature=0.7")
print(f"✓ Memory-optimized config: beam_width={BEAM_WIDTH}, bond_dim={BOND_DIM}, top_k=30, max_length=256")

if COLAB_CONFIG_PATH.exists():
    COLAB_CONFIG_PATH.unlink()

with COLAB_CONFIG_PATH.open("w") as f:
    yaml.safe_dump(config, f)

print(f"Wrote Colab-specific configuration to {COLAB_CONFIG_PATH}")

## 4. Run the Reproduction Script on GPU

This cell executes the main experiment driver using the Colab-friendly configuration. Logs and tables will be stored under `paper_results/`.

In [ ]:
import subprocess

cmd = [
    "python",
    "experiments/reproduce_paper_results.py",
    "--model", MODEL_NAME,
    "--num_examples", str(NUM_EXAMPLES),
    "--alpha", str(ALPHA),
    "--bond_dim", str(BOND_DIM),
    "--config", str(COLAB_CONFIG_PATH),
]

print("Running command:\n", " ".join(cmd))
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)
try:
    for line in process.stdout:
        print(line, end="")
finally:
    return_code = process.wait()

if return_code != 0:
    raise RuntimeError(f"Experiment runner exited with status {return_code}")

## 5. Inspect Results

The reproduction script saves both raw JSON and formatted tables. The cell below displays the latest run summary.

In [ ]:
from pathlib import Path
import json

results_dir = Path("paper_results")
table_files = sorted(results_dir.glob("tables_*.txt"))
if not table_files:
    raise FileNotFoundError("No results found. Ensure the previous cell completed successfully.")

latest_tables = table_files[-1]
print(f"Displaying summary from {latest_tables}")
print(latest_tables.read_text())

json_files = sorted(results_dir.glob("full_results_*.json"))
if json_files:
    latest_json = json_files[-1]
    with latest_json.open("r") as f:
        payload = json.load(f)
    print("\nStored result keys:", list(payload.keys()))
else:
    print("No JSON artifact found.")

## 6. Optional: Interactive Generation

Use TNAD directly for ad-hoc reasoning experiments once the model is loaded into memory.

In [ ]:
from tnad import FidelityGuidedBeamSearcher
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quant_cfg = BitsAndBytesConfig(load_in_8bit=True) if USE_8BIT else None
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto" if quant_cfg else "cuda",
    quantization_config=quant_cfg,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

interactive_searcher = FidelityGuidedBeamSearcher(
    model=model,
    tokenizer=tokenizer,
    beam_width=BEAM_WIDTH,
    alpha=ALPHA,
    bond_dim=BOND_DIM,
)

prompt = "Solve: If a train travels 60 miles in 1.5 hours, what is its average speed in miles per hour?\nAnswer:"
result = interactive_searcher.generate(prompt, max_length=128, min_length=12, return_details=False)
print(result["text"])
